# Pipeline ETL - Projeto de integração de dados - Grupo 06

## Imports e setup

In [1]:
!pip install pandas ipython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 28.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
from IPython.display import HTML
from glob import glob

## Informação dos dados

De [OpenDataSUS Sinan/Dengue](https://opendatasus.saude.gov.br/dataset/arboviroses-dengue):


> O Sistema de Informação de Agravos de Notificação (Sinan) tem como objetivo coletar, transmitir e disseminar dados gerados rotineiramente pela vigilância epidemiológica das três esferas de governo, por meio de uma rede informatizada, para apoiar o processo de investigação e dar subsídios à análise das informações das doenças e dos agravos de notificação compulsória. Atualmente, o sistema possui duas versões vigentes, Sinan Online e Sinan Net.

> O Sinan Online visa à inserção e disseminação dos dados de notificação e investigação de dengue e de febre de chikungunya, enquanto que o Sinan Net é alimentado pela notificação e investigação da grande maioria dos agravos e doenças, que constam na Lista Nacional de Notificação Compulsória de Doenças, Agravos e Eventos de Saúde Pública, do Anexo 1 do Anexo V da Portaria de Consolidação nº 4, de 28 de setembro de 2017, que consolida as normas sobre os sistemas e os subsistemas do Sistema Único de Saúde, mas é facultado a estados e municípios incluir outros problemas de saúde importantes para o seu contexto local.

> Destaca-se que a dengue é doença de notificação compulsória, ou seja, todo caso suspeito e/ou confirmado deve ser obrigatoriamente notificado ao Serviço de Vigilância Epidemiológica da Secretaria Municipal de Saúde (SMS). As notificações de casos suspeitos de dengue devem ser registradas na Ficha de Notificação/Investigação da dengue e chikungunya e inseridas no Sistema de Informação de Agravos de Notificação – Sinan Online. Os óbitos suspeitos pela infecção do vírus dengue (DENV) são de notificação compulsória imediata para todas as esferas de gestão do Sistema Único de Saúde (SUS), a ser realizada em até 24 horas a partir do seu conhecimento, pelo meio de comunicação mais rápido disponível. Posteriormente, os dados devem ser inseridos no Sistema de Informação de Agravos de Notificação (Sinan Online).


Para informações específicas das colunas, consultar o [Dicionário de dados](https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SINAN/Dengue/dic_dados_dengue.pdf).


## Extração dos dados

Baixando os dados. Obtemos os dados de 2020 à 2022 pois o uso de memória é menor que o dos registros mais recentes (e mesmo assim temos por volta de 3G de RAM sendo utilizadas para carregar os dados)

In [4]:
MIN_YEAR = 20
MAX_YEAR = 22

In [5]:
for year in range(MIN_YEAR, MAX_YEAR + 1):
    url = f"https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SINAN/Dengue/csv/DENGBR{year}.csv.zip"
    !wget "$url"
    !unzip -o "DENGBR{year}.csv.zip"
!rm -f *.zip # deleting all zip files

--2025-11-24 09:12:09--  https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SINAN/Dengue/csv/DENGBR20.csv.zip
Resolving s3.sa-east-1.amazonaws.com (s3.sa-east-1.amazonaws.com)... 3.5.233.61, 3.5.232.196, 3.5.233.109, ...
Connecting to s3.sa-east-1.amazonaws.com (s3.sa-east-1.amazonaws.com)|3.5.233.61|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51373250 (49M) [application/zip]
Saving to: ‘DENGBR20.csv.zip’

DENGBR20.csv.zip    100%[===================>]  48.99M  13.7MB/s    in 3.8s    

2025-11-24 09:12:14 (13.0 MB/s) - ‘DENGBR20.csv.zip’ saved [51373250/51373250]

Archive:  DENGBR20.csv.zip
  inflating: DENGBR20.csv            
--2025-11-24 09:12:18--  https://s3.sa-east-1.amazonaws.com/ckan.saude.gov.br/SINAN/Dengue/csv/DENGBR21.csv.zip
Resolving s3.sa-east-1.amazonaws.com (s3.sa-east-1.amazonaws.com)... 3.5.232.210, 3.5.233.89, 3.5.232.4, ...
Connecting to s3.sa-east-1.amazonaws.com (s3.sa-east-1.amazonaws.com)|3.5.232.210|:443... connected.
HTTP reque

Carregando os arquivos individuais e juntando os DataFrames em um só

ATENÇÃO: ALTO USO DE RAM

In [6]:
common_name = "DENGBR{}.csv"

# Start with columns from the first file
sample_path = common_name.format(MAX_YEAR)
master_cols = set(pd.read_csv(sample_path, nrows=0).columns.tolist())

all_dataframes = []

for year in range(MIN_YEAR, MAX_YEAR + 1):
    path = common_name.format(year)
    df = pd.read_csv(path, encoding='utf-8')

    # Find unexpected columns
    unexpected = set(df.columns) - master_cols
    if unexpected:
        print(f"[Warning] {path} has extra columns: {unexpected}")
        # Add them to the master list
        master_cols.update(unexpected)

    all_dataframes.append(df)

# Convert master_cols to ordered list (optional: sort)
master_cols = sorted(master_cols)

# Normalize columns across all dataframes
for i in range(len(all_dataframes)):
    df = all_dataframes[i]
    for col in master_cols:
        if col not in df.columns:
            df[col] = pd.NA
    all_dataframes[i] = df[master_cols]

# Final concatenation
df = pd.concat(all_dataframes, ignore_index=True)

/tmp/ipython-input-2476204088.py:11: DtypeWarning: Columns (22,44,45,46,54,56,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, encoding='utf-8')


[Warning] DENGBR20.csv has extra columns: {'DT_NASC'}


/tmp/ipython-input-2476204088.py:11: DtypeWarning: Columns (22,44,45,46,54,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, encoding='utf-8')
/tmp/ipython-input-2476204088.py:11: DtypeWarning: Columns (22,44,45,46,54,101) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, encoding='utf-8')
/tmp/ipython-input-2476204088.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat(all_dataframes, ignore_index=True)


In [7]:
del all_dataframes

## Transformação dos dados

In [8]:
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3899353 entries, 0 to 3899352
Data columns (total 122 columns):
 #    Column      Non-Null Count    Dtype  
---   ------      --------------    -----  
 0    ACIDO_PEPT  3782453 non-null  float64
 1    ALRM_ABDOM  38132 non-null    float64
 2    ALRM_HEMAT  37975 non-null    float64
 3    ALRM_HEPAT  37959 non-null    float64
 4    ALRM_HIPOT  38083 non-null    float64
 5    ALRM_LETAR  38005 non-null    float64
 6    ALRM_LIQ    37971 non-null    float64
 7    ALRM_PLAQ   38167 non-null    float64
 8    ALRM_SANG   38066 non-null    float64
 9    ALRM_VOM    38043 non-null    float64
 10   ANO_NASC    2389636 non-null  float64
 11   ARTRALGIA   3782453 non-null  float64
 12   ARTRITE     3782453 non-null  float64
 13   AUTO_IMUNE  3782453 non-null  float64
 14   CEFALEIA    3782453 non-null  float64
 15   CLASSI_FIN  3895398 non-null  float64
 16   CLINC_CHIK  22748 non-null    float64
 17   COMPLICA    0 non-null        float64
 18   

In [9]:
total_samples = len(df)

### Verificando quantidade de valores únicos

In [10]:
unique_value_counts = df.nunique(dropna=False)

In [11]:
HTML(unique_value_counts.to_frame("n_unique").to_html(max_rows=None))

,n_unique
ACIDO_PEPT,3
ALRM_ABDOM,3
ALRM_HEMAT,3
ALRM_HEPAT,3
ALRM_HIPOT,3
ALRM_LETAR,3
ALRM_LIQ,3
ALRM_PLAQ,3
ALRM_SANG,3
ALRM_VOM,3


Seguindo o dicionário de dados, `ID_AGRAVO` diz respeito ao código internacional de doenças (CID-10). Na base de dados adquirida, apenas um valor possível é encontrado: A90 (Dengue). Como essa coluna é não-informativa, podemos removê-la para economizar espaço


In [12]:
df.drop("ID_AGRAVO", axis=1, inplace=True)

TODO: Outros elementos possuem apenas um valor úncio mas precisa ser investigado se podem ser removidos também

### Verificando conteúdo dos valores únicos

In [15]:
columns_of_interest = unique_value_counts[unique_value_counts <= 30].index.to_list()
for column in columns_of_interest:
    if column in df.columns:
        print("=" * 10, column, "="*10)
        print(df[column].unique())

========== ACIDO_PEPT ==========
[ 2.  1. nan]
========== ALRM_ABDOM ==========
[nan  2.  1.]
========== ALRM_HEMAT ==========
[nan  2.  1.]
========== ALRM_HEPAT ==========
[nan  2.  1.]
========== ALRM_HIPOT ==========
[nan  2.  1.]
========== ALRM_LETAR ==========
[nan  2.  1.]
========== ALRM_LIQ ==========
[nan  2.  1.]
========== ALRM_PLAQ ==========
[nan  2.  1.]
========== ALRM_SANG ==========
[nan  1.  2.]
========== ALRM_VOM ==========
[nan  2.  1.]
========== ARTRALGIA ==========
[ 2.  1. nan]
========== ARTRITE ==========
[ 2.  1. nan]
========== AUTO_IMUNE ==========
[ 2.  1. nan]
========== CEFALEIA ==========
[ 2.  1. nan]
========== CLASSI_FIN ==========
[ 5. 10.  8. 11. 12. nan]
========== CLINC_CHIK ==========
[nan  1.  2.]
========== COMPLICA ==========
[nan]
========== CONJUNTVIT ==========
[ 2.  1. nan]
========== CON_FHD ==========
[nan]
========== COPAISINF ==========
[ nan   1.  31.  44. 126.  68. 140.  32. 186.  21. 129.  57.  72. 156.
 111.]
========== COUFINF

TODO: Cruzar informações com dicionário de dados para checar se há colunas redundantes ou possíveis simplificações

### Verificando porcentagem de valores nulos

In [16]:
null_information = df.isnull().mean()

In [18]:
HTML((null_information * 100).to_frame("perc_null").sort_values("perc_null").to_html(max_rows=None, float_format="%.6f"))

,perc_null
DT_NOTIFIC,0.000000
NU_IDADE_N,0.000000
NU_ANO,0.000000
SG_UF,0.000000
TP_NOT,0.000000
ID_MUNICIP,0.000000
SG_UF_NOT,0.000000
SEM_PRI,0.000026
ID_PAIS,0.000308
CS_SEXO,0.000949


TODO: Verificar se algum dos 100% nulos podem ser removidos

## Load (Carregamento) dos dados